In [2]:
import pandas as pd
import numpy as np
from rouge_score import rouge_scorer, scoring
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification,AutoModelForSeq2SeqLM
# from transformers import PegasusForConditionalGeneration, PegasusTokenizer
# from transformers import AutoModelForCausalLM
from tqdm import tqdm
import json

#load bert score model
from evaluate import load
bert_score = load("bertscore")


def rouge(refs, preds):
    """
    Returns `t5` style ROUGE scores. See the related implementation:
    https://github.com/google-research/text-to-text-transfer-transformer/blob/3d10afd51ba97ac29eb66ae701eca274488202f7/t5/evaluation/metrics.py#L68
    :param refs:
        A `list` of reference `strs`.
    :param preds:
        A `list` of predicted `strs`.
    """
    rouge_types = ["rouge1", "rouge2", "rougeLsum"]
    scorer = rouge_scorer.RougeScorer(rouge_types)
    # Add newlines between sentences to correctly compute `rougeLsum`.

    def _prepare_summary(summary):
        summary = summary.replace(" . ", ".\n")
        return summary

    # Accumulate confidence intervals.
    aggregator = scoring.BootstrapAggregator()
    for ref, pred in zip(refs, preds):
        ref = _prepare_summary(ref)
        pred = _prepare_summary(pred)
        aggregator.add_scores(scorer.score(ref, pred))
    result = aggregator.aggregate()
    
    return {type: result[type].mid.fmeasure  for type in rouge_types}


def BertScore(refs, preds):
    bert_score_res = bert_score.compute(predictions=[refs], references=[preds], model_type="microsoft/deberta-xlarge-mnli", lang="en")
    
    return bert_score_res

def get_score(refs, preds, acticles):
    rouge_res = rouge([refs], [preds])
    
    bert_score_res = BertScore(refs, preds)
    
    total_res = {
            "rouge1": rouge_res["rouge1"],
            "rougeL": rouge_res["rougeLsum"],
            "bertscore_f1": float(bert_score_res["f1"][0])
        }
    
    return total_res

In [35]:
data = pd.read_json("/home/xbr/LLM/benchmark_llm_summarization/likert_evaluation_results.json")

reference = data[data['model']=="reference"].drop_duplicates(subset=['article'])

davinci_0shot = data[data['model']=="openai_text-davinci-002 (0 shot)"].drop_duplicates(subset=['article'])



writer = pd.read_json("/home/xbr/LLM/benchmark_llm_summarization/writer_summaries.json").to_dict(orient='records')

for i in range(len(writer)):
    news = writer[i]['article'][:20]
    print(news)
    l = reference.index[reference['article'].str.contains(news)][0]
    
    
    



Nick Scholfield is l
Nick Scholfield is l
Nick Scholfield is l
Dr Mehmet Oz's fello
Dr Mehmet Oz's fello
"It's all about taki


IndexError: index 0 is out of bounds for axis 0 with size 0